In [6]:
# Extract stream tables from family planning annual reports
# Reports from the government Office of Population Affairs
# 2003- 2023

In [337]:
import tabula.io as t
import pdfplumber
from PyPDF2 import PdfWriter, PdfReader
import os
import time
import re
os.chdir("/Users/kanyin/Documents/Kanyin/Data Science/Data Projects/STI/Other")

In [37]:
os.listdir() #All pdfs were in a separate folder. Check that no other files except for pdfs

In [100]:
pdf_list=os.listdir()
pdf_list = [x for x in pdf_list if not x.startswith(".")] #removing hidden files

In [101]:
search_terms=["Number of female family planning users by primary contraceptive method and age:",
              "Number of male family planning users by primary contraceptive method and age:",
             "Number of female family planning users, by primary contraceptive method and age:",
             "Number of male family planning users, by primary contraceptive method and age: "]

In [8]:
def search_pages(pdfs, search_terms):
    start_time = time.time()

    """
    Finds pages in a PDF that contain specified search terms and stores them in a global list.

    Parameters:
    - pdf_list (list of str): List of PDF file paths.
    - search_terms (str or list): Keyword(s) to search for (case-insensitive).

    Returns:
     - a dictionary: {pdf_filename: [list of matching_page_numbers]}
    """
    global ref


    if isinstance(search_terms, str):
        search_terms = [search_terms]

    ref={}
    for year in pdfs:
        print(f"Reading page {year}")
        matching_pages=[]
        with pdfplumber.open(year) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text and any(term.lower() in text.lower() for term in search_terms):
                    matching_pages.append(i + 1)
                    matching_pages=[num for num in matching_pages if num >= 30] #remove table of contents
                    name=f"{year}"
                    ref[name]=matching_pages
        time.sleep(2) # Simulate some work
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Iteration {i+1}: {elapsed_time:.4f} seconds")

    return ref

In [235]:
search_pages(pdf_list, search_terms)

Reading page fpar-2010-national-summary.pdf
Iteration 113: 96.8973 seconds
Reading page title-x-fpar-2016-national.pdf
Iteration 139: 173.3847 seconds
Reading page title-x-fpar-2017-national-summary.pdf
Iteration 136: 241.3666 seconds
Reading page 2022-FPAR-National-Summary.pdf
Iteration 172: 371.5553 seconds
Reading page fpar-2007-national-summary.pdf
Iteration 108: 446.9910 seconds
Reading page 2023-FPAR-National-Summary-Report.pdf
Iteration 122: 545.2640 seconds
Reading page fpar-2008-national-summary.pdf
Iteration 111: 578.1383 seconds
Reading page title-x-fpar-2018-national-summary.pdf
Iteration 136: 617.7540 seconds
Reading page 2021-fpar-national-final-508.pdf
Iteration 168: 665.3677 seconds
Reading page fpar-2011-national-summary.pdf
Iteration 115: 707.5338 seconds
Reading page fpar-2009-national-summary.pdf
Iteration 114: 756.3453 seconds
Reading page fpar-national-summary-2012.pdf
Iteration 129: 806.8566 seconds
Reading page title-x-fpar-2019-national-summary.pdf
Iteration 13

{'fpar-2010-national-summary.pdf': [39, 43],
 'title-x-fpar-2016-national.pdf': [44, 50],
 'title-x-fpar-2017-national-summary.pdf': [44, 50],
 '2022-FPAR-National-Summary.pdf': [52, 56],
 'fpar-2007-national-summary.pdf': [40, 44],
 '2023-FPAR-National-Summary-Report.pdf': [64, 69],
 'fpar-2008-national-summary.pdf': [40, 44],
 'title-x-fpar-2018-national-summary.pdf': [44, 50],
 '2021-fpar-national-final-508.pdf': [50, 54],
 'fpar-2011-national-summary.pdf': [40, 44],
 'fpar-2009-national-summary.pdf': [40, 44],
 'fpar-national-summary-2012.pdf': [45, 49],
 'title-x-fpar-2019-national-summary.pdf': [46, 50],
 'fpar-2006-national-summary.pdf': [38, 42],
 'fpar-2005-national-summary.pdf': [40, 44],
 'title-x-fpar-2020-national-summary-sep-2021.pdf': [52, 56],
 'title-x-fpar-2015.pdf': [46, 52],
 'fpar-2013-national-summary.pdf': [44, 48],
 'title-x-fpar-2014-national.pdf': [46, 52]}

In [301]:
all_tables={}
def get_tables(pages):
    start_time = time.time()
    """
    Takes dictionary of matching pages and parses them to return tables.
    """
    for i, pdf in enumerate(ref):
        
        for input_pdf, matching_pages in ref.items(): #to clean up names 
                input_pdf_lower = input_pdf.lower()
                has_fpar = "fpar" in input_pdf_lower
                year_match = re.search(r'20\d{2}', input_pdf_lower)
            
                if has_fpar and year_match:
                    year = year_match.group()
                    name = f"fpar-{year}"

                
                for i, number in enumerate(matching_pages):
                    print(f"Reading page {number} from '{input_pdf}'...")
        
                    table = t.read_pdf(
                        input_pdf,
                        pages=number,
                        multiple_tables=True,
                        stream=True,
                        output_format="dataframe",
                        guess=True
                    )
                        
                    key= f"{name}_page{number}_table{i+1}"
                    all_tables[key] = table
                    time.sleep(2) 
        
                    print(f"Completed reading page {number} from '{name}'. Found {len(tables)} tables.")
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    print(f"Time lapsed: {elapsed_time:.4f} seconds")
                    
        return all_tables
    

In [348]:
get_tables(ref)

Reading page 39 from 'fpar-2010-national-summary.pdf'...
Completed reading page 39 from 'fpar-2010'. Found 1 tables.
Time lapsed: 7.3980 seconds
Reading page 43 from 'fpar-2010-national-summary.pdf'...
Completed reading page 43 from 'fpar-2010'. Found 1 tables.
Time lapsed: 11.8856 seconds
Reading page 44 from 'title-x-fpar-2016-national.pdf'...
Completed reading page 44 from 'fpar-2016'. Found 1 tables.
Time lapsed: 16.7341 seconds
Reading page 50 from 'title-x-fpar-2016-national.pdf'...
Completed reading page 50 from 'fpar-2016'. Found 1 tables.
Time lapsed: 21.4741 seconds
Reading page 44 from 'title-x-fpar-2017-national-summary.pdf'...
Completed reading page 44 from 'fpar-2017'. Found 1 tables.
Time lapsed: 26.1869 seconds
Reading page 50 from 'title-x-fpar-2017-national-summary.pdf'...
Completed reading page 50 from 'fpar-2017'. Found 1 tables.
Time lapsed: 30.6638 seconds
Reading page 52 from '2022-FPAR-National-Summary.pdf'...
Completed reading page 52 from 'fpar-2022'. Found 1 

Got stderr: Jun 18, 2025 7:28:57 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 18, 2025 7:28:58 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Completed reading page 40 from 'fpar-2007'. Found 1 tables.
Time lapsed: 44.4855 seconds
Reading page 44 from 'fpar-2007-national-summary.pdf'...


Got stderr: Jun 18, 2025 7:29:02 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 18, 2025 7:29:02 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Completed reading page 44 from 'fpar-2007'. Found 1 tables.
Time lapsed: 48.5300 seconds
Reading page 64 from '2023-FPAR-National-Summary-Report.pdf'...
Completed reading page 64 from 'fpar-2023'. Found 1 tables.
Time lapsed: 53.1448 seconds
Reading page 69 from '2023-FPAR-National-Summary-Report.pdf'...
Completed reading page 69 from 'fpar-2023'. Found 1 tables.
Time lapsed: 57.6830 seconds
Reading page 40 from 'fpar-2008-national-summary.pdf'...
Completed reading page 40 from 'fpar-2008'. Found 1 tables.
Time lapsed: 61.8268 seconds
Reading page 44 from 'fpar-2008-national-summary.pdf'...
Completed reading page 44 from 'fpar-2008'. Found 1 tables.
Time lapsed: 65.9450 seconds
Reading page 44 from 'title-x-fpar-2018-national-summary.pdf'...
Completed reading page 44 from 'fpar-2018'. Found 1 tables.
Time lapsed: 70.4280 seconds
Reading page 50 from 'title-x-fpar-2018-national-summary.pdf'...
Completed reading page 50 from 'fpar-2018'. Found 1 tables.
Time lapsed: 74.7781 seconds
Readi

Got stderr: Jun 18, 2025 7:30:33 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jun 18, 2025 7:30:34 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile



Completed reading page 52 from 'fpar-2020'. Found 1 tables.
Time lapsed: 140.1658 seconds
Reading page 56 from 'title-x-fpar-2020-national-summary-sep-2021.pdf'...


Got stderr: Jun 18, 2025 7:30:37 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jun 18, 2025 7:30:38 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile



Completed reading page 56 from 'fpar-2020'. Found 1 tables.
Time lapsed: 144.6894 seconds
Reading page 46 from 'title-x-fpar-2015.pdf'...
Completed reading page 46 from 'fpar-2015'. Found 1 tables.
Time lapsed: 149.3109 seconds
Reading page 52 from 'title-x-fpar-2015.pdf'...
Completed reading page 52 from 'fpar-2015'. Found 1 tables.
Time lapsed: 153.7202 seconds
Reading page 44 from 'fpar-2013-national-summary.pdf'...
Completed reading page 44 from 'fpar-2013'. Found 1 tables.
Time lapsed: 158.0959 seconds
Reading page 48 from 'fpar-2013-national-summary.pdf'...
Completed reading page 48 from 'fpar-2013'. Found 1 tables.
Time lapsed: 162.2897 seconds
Reading page 46 from 'title-x-fpar-2014-national.pdf'...
Completed reading page 46 from 'fpar-2014'. Found 1 tables.
Time lapsed: 166.8185 seconds
Reading page 52 from 'title-x-fpar-2014-national.pdf'...
Completed reading page 52 from 'fpar-2014'. Found 1 tables.
Time lapsed: 171.2398 seconds


{'fpar-2010_page39_table1': [              Oral contraceptive  1,684,201    18,390   168,433   229,392  \
  0            Contraceptive patch     93,499     1,241     9,882    12,278   
  1                   Vaginal ring    186,238       717    11,581    22,178   
  2         Cervical cap/diaphragm      4,402         7       148       190   
  3           Contraceptive sponge      1,581        17       150       211   
  4                  Female condom      5,944        23       372       618   
  5        Spermicide (used alone)      8,346        46       470       645   
  6   Fertility awareness method b     14,379        72       424       868   
  7                   Abstinence c     75,534     9,091    11,126     7,005   
  8                 Other method d    116,635     1,014     8,151    11,729   
  9               Method unknown e    160,788     2,496    12,498    16,740   
  10           Rely on Male Method        NaN       NaN       NaN       NaN   
  11                     

In [351]:
# Save the list of DataFrames using pickle
import pandas as pd
import pickle
with open('list_of_dataframes.pkl', 'wb') as f:
    pickle.dump(all_tables, f)

In [352]:
with open('list_of_dataframes.pkl', 'rb') as f:
    loaded_list_of_dfs = pickle.load(f)

In [353]:
loaded_list_of_dfs

{'fpar-2010_page39_table1': [              Oral contraceptive  1,684,201    18,390   168,433   229,392  \
  0            Contraceptive patch     93,499     1,241     9,882    12,278   
  1                   Vaginal ring    186,238       717    11,581    22,178   
  2         Cervical cap/diaphragm      4,402         7       148       190   
  3           Contraceptive sponge      1,581        17       150       211   
  4                  Female condom      5,944        23       372       618   
  5        Spermicide (used alone)      8,346        46       470       645   
  6   Fertility awareness method b     14,379        72       424       868   
  7                   Abstinence c     75,534     9,091    11,126     7,005   
  8                 Other method d    116,635     1,014     8,151    11,729   
  9               Method unknown e    160,788     2,496    12,498    16,740   
  10           Rely on Male Method        NaN       NaN       NaN       NaN   
  11                     

In [360]:
import re
import pandas as pd

rates = {}
redo = {}

def crude_rates(list_tables):
    for i, (string, dataset) in enumerate(list_tables.items(), start=1):
        table = dataset[0].copy()
        print(table)

        answer = input("Continue? (y/n): ") #optional step to look at the table to make pdf sure the scrape was successful
        if answer.lower() == 'n':
            redo[i] = string
            continue

        name = f'{string}'
        sex = 'Female' if 'table1' in name.lower() else 'Male'
        year_match = re.search(r'20\d{2}', string)
        year = year_match.group() if year_match else 'unknown'
        updated_name = f'fpar-{year}-{sex}'

        # Set index to first column
        table.set_index(table.columns[0], inplace=True)
        table.index = table.index.astype(str)
        firstcol = table.columns[0]

        # Get index labels
        try:
            Male = table.index[table.index.str.match('Male condom', case=False)][0]
            Abs = table.index[table.index.str.contains('Abstin', case=False)][0]
            Tot = table.index[table.index.str.contains('Total', case=False)][0]
        except IndexError:
            print(f"⚠️ Missing required row (Male/Abstinence/Total) in: {string}")
            redo[i] = string
            continue

        # Female condom is optional
        Fem_index = table.index[table.index.str.match('Female condom', case=False)]
        Fem = Fem_index[0] if not Fem_index.empty else None

        # Helper function to clean and convert value
        def get_int_value(index_label):
            val = table.loc[index_label, firstcol]
            if isinstance(val, pd.Series):
                val = val.iloc[0]
            return int(str(val).replace(',', '').strip())

        try:
            fem_value = get_int_value(Fem) if Fem else 0
            male_value = get_int_value(Male)
            abst_value = get_int_value(Abs)
            total_value = get_int_value(Tot)
        except Exception as e:
            print(f"⚠️ Value conversion failed in {string}: {e}")
            redo[i] = string
            continue

        # Compute results
        cndm_value = male_value if string.endswith('2') else male_value + fem_value
        activ_value = total_value - abst_value
        crude_rate = cndm_value / activ_value * 100000 if activ_value > 0 else 0

        result = {
            'condom_users': cndm_value,
            'active_population': activ_value,
            'crude_rate': crude_rate
        }

        rates[updated_name] = result
        print(f"✅ {updated_name}: {result}")

    return rates


In [361]:
crude_rates(all_tables)

              Oral contraceptive  1,684,201    18,390   168,433   229,392  \
0            Contraceptive patch     93,499     1,241     9,882    12,278   
1                   Vaginal ring    186,238       717    11,581    22,178   
2         Cervical cap/diaphragm      4,402         7       148       190   
3           Contraceptive sponge      1,581        17       150       211   
4                  Female condom      5,944        23       372       618   
5        Spermicide (used alone)      8,346        46       470       645   
6   Fertility awareness method b     14,379        72       424       868   
7                   Abstinence c     75,534     9,091    11,126     7,005   
8                 Other method d    116,635     1,014     8,151    11,729   
9               Method unknown e    160,788     2,496    12,498    16,740   
10           Rely on Male Method        NaN       NaN       NaN       NaN   
11                     Vasectomy      8,683         0         1        65   

Continue? (y/n):  y


✅ fpar-2010-Female: {'condom_users': 793273, 'active_population': 4747036, 'crude_rate': 16710.911819501685}
                            Unnamed: 0  All Age Under 15 15 to 17 18 to 19  \
0                       Primary Method   Groups    Years    Years    Years   
1                            Vasectomy    4,676        0        0        0   
2                          Male condom  282,672    4,028   24,343   33,115   
3         Fertility awareness method a      768        5       47       37   
4                         Abstinence b   23,243    9,439    6,009    1,478   
5                       Other method c    9,983      248      781      815   
6                     Method unknown d   19,677      714    2,529    1,762   
7              Rely on female method e   35,606      237    1,224    2,719   
8                            No Method      NaN      NaN      NaN      NaN   
9   Partner pregnant/seeking pregnancy    3,630       25      175      260   
10                        Other r

Continue? (y/n):  


✅ fpar-2010-Male: {'condom_users': 282672, 'active_population': 379049, 'crude_rate': 74573.99966758916}
                 Oral contraceptive    946,383   7,322   76,156  111,830  \
0               Contraceptive patch     47,030     418    3,841    5,352   
1                      Vaginal ring     83,473     181    3,220    6,249   
2         Cervical cap or diaphragm      2,130      17      188      158   
3              Contraceptive sponge        138       2        7        6   
4                     Female condom      2,929      17      189      252   
5           Spermicide (used alone)      1,848      23      119      134   
6                       FAM or LAMb     14,392     103      453      764   
7                       Abstinencec     89,102  11,445   12,678    6,392   
8       Withdrawal or other methodd     75,191     944    3,215    5,268   
9               Rely on Male Method        NaN     NaN      NaN      NaN   
10                        Vasectomy      8,178       0     

...

Continue? (y/n):  


✅ fpar-2014-Female: {'condom_users': 581447, 'active_population': 3694524, 'crude_rate': 15738.076136465754}
                            Unnamed: 0  All Age Under 15 15 to 17 18 to 19  \
0                       Primary Method   Groups    Years    Years    Years   
1                            Vasectomy    2,763        0        0        0   
2                          Male condom  262,255    1,706   14,337   22,050   
3                                FAM a    1,079        6       29       56   
4                         Abstinence b   21,127    5,512    4,974    1,246   
5         Withdrawal or other method c    9,992      149      675      633   
6              Rely on female method d   22,063      167      761    1,298   
7                            No Method      NaN      NaN      NaN      NaN   
8   Partner pregnant/seeking pregnancy    3,253       36      165      168   
9                         Other reason   21,501      771    1,696    1,415   
10                    Method Unkn

Continue? (y/n):  


✅ fpar-2014-Male: {'condom_users': 262255, 'active_population': 343534, 'crude_rate': 76340.33312568771}


{'fpar-2010-Female': {'condom_users': 793273,
  'active_population': 4747036,
  'crude_rate': 16710.911819501685},
 'fpar-2010-Male': {'condom_users': 282672,
  'active_population': 379049,
  'crude_rate': 74573.99966758916},
 'fpar-2016-Female': {'condom_users': 562285,
  'active_population': 3463916,
  'crude_rate': 16232.639590567438},
 'fpar-2016-Male': {'condom_users': 297265,
  'active_population': 422070,
  'crude_rate': 70430.26038334874},
 'fpar-2017-Female': {'condom_users': 549666,
  'active_population': 3448850,
  'crude_rate': 15937.660379546805},
 'fpar-2017-Male': {'condom_users': 299268,
  'active_population': 429736,
  'crude_rate': 69639.96500176852},
 'fpar-2022-Female': {'condom_users': 322162,
  'active_population': 2118361,
  'crude_rate': 15208.078320928304},
 'fpar-2022-Male': {'condom_users': 175568,
  'active_population': 327910,
  'crude_rate': 53541.520539172336},
 'fpar-2007-Female': {'condom_users': 720571,
  'active_population': 4637870,
  'crude_rate': 1

In [371]:
os.getcwd()

'/Users/kanyin/Documents/Kanyin/Data Science/Data Projects/STI/Other'

In [372]:
rates_table=pd.DataFrame(rates).transpose().sort_index()
rates_table.to_csv("/Users/kanyin/Documents/Kanyin/Data Science/Data Projects/STI/Data/STI_Protection.csv")